### Run this on a terminal to create a custom environment:<br>
bash<br>
conda create --name fiftyone python=3.8 -y<br>
conda activate fiftyone<br>
conda install ipykernel -y<br>
pip install setuptools==59.8.0

# Clone master

In [ ]:
!git clone https://github.com/voxel51/fiftyone.git ./fiftyone --depth 1

In [ ]:
# remove
!rm -rf ./fiftyone

# Setup

## Fix values

In [ ]:
import json
log_path = '/opt/ml/metadata/resource-metadata.json'
with open(log_path, 'r') as logs:
    _logs = json.load(logs)
INSTANCE_NAME = _logs['ResourceName']
F1_HOST = f"https://{INSTANCE_NAME}.notebook.eu-central-1.sagemaker.aws"
F1_PORT = 6008
%env F1_HOST={F1_HOST}
%env F1_PORT={F1_PORT}

In [ ]:
%%bash
cd ./fiftyone/app/packages/app/
if [ -f ".env" ] ; then
    rm ".env"
fi
if [ -f ".env.development" ] ; then
    rm ".env.development"
fi
if [ -f ".env.development.local" ] ; then
    rm ".env.development.local"
fi
echo "VITE_API=$F1_HOST/proxy/$F1_PORT" >> .env
cat .env
cd ../../../../

In [ ]:
# Fix package.json
old_value = 'vite build",'
new_value = f'vite build --base=/proxy/{F1_PORT}/",'
!sed -i 's+{old_value}+{new_value}+g' ./fiftyone/app/packages/app/package.json

In [ ]:
!cat fiftyone/app/packages/app/package.json

In [ ]:
# Fix context.py
old_value = 'http://{address}:{port}/'
new_value = f'{F1_HOST}/proxy/{F1_PORT}/'
!sed -i "s+{old_value}+{new_value}+g" ./fiftyone/fiftyone/core/context.py

In [ ]:
# Fix context.py
old_value = '?context='
new_value = '?polling=true\&context='
!sed -i "s+{old_value}+{new_value}+g" ./fiftyone/fiftyone/core/context.py

In [ ]:
!cat ./fiftyone/fiftyone/core/context.py

## Build NodeJS app

In [ ]:
!chmod +x ./install.bash
!./install.bash -p

## Install

In [ ]:
!chmod +x ./install.bash
!./install.bash -e

# Launch

In [ ]:
%env FIFTYONE_DATABASE_URI=mongodb://127.0.0.1:27017

In [ ]:
import fiftyone as fo

In [ ]:
fo.config

In [ ]:
# Remove old dataset if existing
if "COCO2017" in fo.list_datasets():
    dataset = fo.load_dataset("COCO2017")
    dataset.delete()
dataset = fo.Dataset.from_dir(
    dataset_dir="/tmp/bucket/dataset/images_classified_grouped_V2/A5/",
    dataset_type=fo.types.ImageDirectory,
    name="COCO2017",
)

In [ ]:
from IPython.core.display import display, HTML
from uuid import uuid4

session = fo.launch_app(dataset, port=6008, auto=False)

uuid = str(uuid4())
print(uuid)
URL = session.url + f"?subscription={uuid}&polling=true"
display(
    HTML(
        """
        <a href="{}" target="_blank"><h1>Open session in a new tab</h1></a>
        """.format(URL)
    )
)

In [ ]:
# Run this to use the app here
session.show()

### Start server manually

In [ ]:
%cd fiftyone

In [ ]:
!python ./fiftyone/server/main.py --port 6008